In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as eptl
from osgeo import gdal
import rasterio
import pandas as pd


data = et.data.get_data('cold-springs-fire')
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

Extracted output to /root/earth-analytics/data/cold-springs-fire/.


In [ ]:
naip_data_path = os.path.join("cold-springs-fire",
                              "naip",
                              "m_3910505_nw_13_1_20150919",
                              "crop",
                              "m_3910505_nw_13_1_20150919_crop.tif")

naip_data = rxr.open_rasterio(naip_data_path)

naip_data.shape

(4, 2312, 4377)

In [ ]:
band_nir_tif = "/content/drive/MyDrive/CollabResources/Tiff/Photos/IMG_0005_4.tif"
band_red_tif = "/content/drive/MyDrive/CollabResources/Tiff/Photos/IMG_0005_3.tif"
reflection = np.genfromtxt('/content/drive/MyDrive/CollabResources/Tiff/RP04-1923118-OB.csv', delimiter=',')

In [ ]:
def tif_to_array(tif_path):
    tif_dataset = gdal.Open(tif_path)
    # print(tif_dataset)

    raster_array = tif_dataset.ReadAsArray()

    return raster_array

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CollabResources/Tiff/RP04-1923118-OB.csv')

reflectance_values = df.iloc[:, 1].values

pixel_values = np.arange(250, 951)

def calibrate_reflectance(pixel_values, reflectance_values, pixel_array):
    # Linear interpolation
    calibrated_reflectance = np.interp(pixel_array, pixel_values, reflectance_values)
    return calibrated_reflectance

In [ ]:
nir_array = tif_to_array(band_nir_tif)
red_array = tif_to_array(band_red_tif)
# nir_array = nir_dataset.ReadAsArray()

calibrated_nir = calibrate_reflectance(pixel_values, reflectance_values, nir_array)
calibrated_red = calibrate_reflectance(pixel_values, reflectance_values, red_array)

naip_ndvi = es.normalized_diff(nir_array, red_array)

AttributeError: 'NoneType' object has no attribute 'ReadAsArray'

In [ ]:
ep.plot_bands(naip_ndvi,
              cmap='PiYG',
              scale=False,
              vmin=-1, vmax=1,
              title="Derived NDVI")
plt.show()

NameError: name 'ep' is not defined

In [ ]:
print(naip_ndvi)
# ep.hist(naip_ndvi,
#         figsize=(12, 6),
#         title=["NDVI: Distribution of pixels\n NAIP 2015 Cold Springs fire site"])

# plt.show()

[[0.0650730411686587 0.1394736842105263 0.16853932584269662 ...
  0.6471399035148173 0.6281690140845071 0.6273551988834613]
 [0.10662824207492795 0.11587982832618025 0.1584852734922861 ...
  0.6325385694249649 0.6238805970149254 0.6018882769472856]
 [0.08166409861325115 0.11411411411411411 0.09969788519637462 ...
  0.580146460537022 0.5908720456397718 0.5902834008097166]
 ...
 [0.4524959742351047 0.45563139931740615 0.49829351535836175 ...
  0.5656268393172454 0.5426170468187275 0.5283687943262412]
 [0.4266777133388567 0.4483306836248013 0.48026315789473684 ...
  0.5614973262032086 0.5308938212357528 0.5205724508050089]
 [0.4768976897689769 0.500823723228995 0.47351263243683783 ...
  0.5591016548463357 0.5104808877928483 0.5141651597347799]]


In [ ]:
type(naip_ndvi), naip_ndvi.dtype

(xarray.core.dataarray.DataArray, dtype('float64'))

In [ ]:
naip_ndvi_outpath = os.path.join("cold-springs-fire",
                                 "outputs",
                                 "naip_ndvi.tif")

# Write your the ndvi raster object
naip_ndvi.rio.to_raster(naip_ndvi_outpath)